# Scipp widgets examples

The scipp-widgets module contains some simple wrapper widgets to aid in creating graphical notebooks.
This allows arguments to wrapped functions to be displayed and selected graphically.
To do this it builds upon the [ipywidgets](https://ipywidgets.readthedocs.io/) module.




For demonstration purposes we first create a simple set of [scipp](https://scipp.github.io/tutorials/multi-d-datasets.html) data.

In [2]:
import numpy as np
import scipp as sc
d = sc.Dataset(
    {
    'alice': sc.Variable(['z', 'y', 'x'], values=np.random.rand(10, 10, 10),
                         variances=0.1*np.random.rand(10, 10, 10)),
    'bob': sc.Variable(['x', 'z'], values=np.arange(0.0, 10.0, 0.1).reshape(10, 10),
                       variances=0.1*np.random.rand(10, 10))
    },
    coords={
        'x': sc.Variable(['x'], values=np.arange(11.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(11.0), unit=sc.units.m),
        'z': sc.Variable(['z'], values=np.arange(11.0), unit=sc.units.m)})

## Wrapping a processing function
scipp-widgets allows functions to be wrapped in simple graphical interfaces.
Taking inputs from, and returning outputs to, the notebook's global scope.
As a first example, take the `scipp.sum` function.

We will need to create an input specification object for this describing how we want it displayed, what validation we wish to perform and which argument of the underlying function it should correspond to.

In [ ]:
from scipp_widgets.input_spec import InputSpec, StringInputSpec
data_input = InputSpec('x', tooltip='input data')
dimension_input = StringInputSpec('dim', options = ('x', 'z'))

The `eval_input` allows variables in the notebooks global scope to be passed to this input by name.

A widget which wraps the plot function can then be constructed like so:

In [ ]:
from scipp_widgets.widgets import ProcessWidget
ProcessWidget(sc.sum, [data_input, dimension_input])

## Wrapping a display function

Another common case is to wrap a function and display its result without adding it to the notebooks global scope.
As an example, take the `scipp.plot.plot` function. This takes a scipp object and plots it. 

In [ ]:
from scipp_widgets.widgets import DisplayWidget
data_input = InputSpec('scipp_obj')
DisplayWidget(sc.plot.plot, [data_input])

## Validators
Validator methods can be added to any input. 
These are run when the widget is processing and serve to both perform any pre-processing the input needs before it is passed to the underlying function as well as perform any validation specified.

For example we could add a validator to our wrapped plot function meaning it will only accept scipp objects as inputs.

In [ ]:
from scipp_widgets.validators import scipp_object_validator
data_input = InputSpec('scipp_obj', validator=scipp_object_validator, tooltip='Data to plot')
DisplayWidget(sc.plot.plot, [data_input])

## Hiding code
The code block used to create a widget can be hidden. Click on the `Py` button to toggle the code back to visibility.

In [ ]:
data_input = InputSpec('scipp_obj', validator=scipp_object_validator, tooltip='Data to plot')
DisplayWidget(sc.plot.plot, [data_input], hide_code=True)

## Examples

### linked scipp object and dimension input

In [ ]:
from scipp_widgets.input_spec import ScippInputWithDimSpec
data_dim_input = ScippInputWithDimSpec(['x', 'dim'])
ProcessWidget(sc.sum, [data_dim_input])

In [16]:
from inspect import signature, Parameter
signature(sc.sum).parameters['x'].default == Parameter.empty

True

In [ ]:
from dataclasses import dataclass
from typing import Any

@dataclass
class FuncParameter(repr=False):
    name: str
    default: Any
    as_string: str
        
    def __repr__(self):
        return as_string
    
    

def parse_signature(func: Callable):
    parameters = []
    for key, value in signature(func).items():
        parameters.append(FuncParameter(key, value.default, value., value.))
        